<a href="https://colab.research.google.com/github/thomasjngo/Real-Estate-PySpark/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [23]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [25]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)

home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [36]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import avg, round

four_bedroom_houses = home_sales_df.filter(home_sales_df.bedrooms == 4)

# Group the data by year and calculate the average price for each year
average_price_per_year = four_bedroom_houses.groupBy('date_built').avg('price')

# Round the average prices to two decimal places
average_price_per_year = average_price_per_year.withColumn('avg(price)', round('avg(price)', 2))

# Show the results
average_price_per_year.show()

+----------+----------+
|date_built|avg(price)|
+----------+----------+
|      2015| 307908.86|
|      2013| 299999.39|
|      2014| 299073.89|
|      2012| 298233.42|
|      2016| 296050.24|
|      2010| 296800.75|
|      2011|  302141.9|
|      2017| 296576.69|
+----------+----------+



In [30]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

filtered_df = home_sales_df.filter((home_sales_df["bedrooms"] == 3) & (home_sales_df["bathrooms"] == 3))

# Group the data by the year the home was built and calculate the average price
average_price_df = filtered_df.groupBy("date_built").avg("price")

# Round the average price to two decimal places
average_price_rounded_df = average_price_df.withColumn("avg(price)", round("avg(price)", 2))

# Show the results
average_price_rounded_df.show()

+----------+----------+
|date_built|avg(price)|
+----------+----------+
|      2015|  288770.3|
|      2013| 295962.27|
|      2014| 290852.27|
|      2012| 293683.19|
|      2016| 290555.07|
|      2010| 292859.62|
|      2011| 291117.47|
|      2017| 292676.79|
+----------+----------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


In [ ]:
# 11. Read the parquet formatted data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

